In [1]:
!pip install transformers torch scikit-learn py-rouge
!pip install sentence-transformers
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a84774f5f2d2aba5d0823f434d3aa821de0be0fc0b90c74a6af91e7ec7a47b82
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/de/ea/025db0a39337b63d4728a900d262c39c3029b0fe76a9876ce6297b1aa6a0/sacrebleu-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.4 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-no

In [49]:
import json
import os
import re
import pandas as pd
from html import unescape
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import get_cosine_schedule_with_warmup
import numpy as np
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from datasets import load_dataset
import langid
from tqdm import tqdm
import time

In [8]:
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize
from nltk import ngrams
from nltk.chunk import ne_chunk
from nltk.tree import Tree
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


True

# Xử lý file từ Hugging Face

In [4]:
dataset = load_dataset("cc_news")

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
        num_rows: 708241
    })
})

In [6]:
dataset['train']['title']

['Daughter Duo is Dancing in The Same Company',
 'New York City Ballet Announces Interim Leadership Team',
 'Watch Pennsylvania Ballet & Boston Ballet Face Off for the Super Bowl',
 'dance shoes',
 'Rebecca Krohn on Her Retirement from New York City Ballet',
 "Roy Kaiser to Become Nevada Ballet Theatre's New Artistic Director",
 "What It's Like Inside NYCB After Peter Martins",
 'Nutcracker Secrets and Surprises',
 'Inside the Beijing Dance Academy',
 'dance shoes',
 'Isabella Boylston and James Whiteside Get Hilariously Candid',
 'Ballet Performances This Week',
 'Guillaume Côté on NBoC\'s "Frame by Frame"',
 'Broadway\'s "Carousel" Stars Some Familiar Ballet Faces',
 'The Joffrey Presents Ekman\'s "Midsummer Night\'s Dream"',
 'Wonderfully Simple Graphic Design Software',
 'Bay Community News',
 '25 Year Old Molests Child Under 12',
 'How to install the Trac project management tool on Ubuntu 16.04',
 'New Amazon class certifies cloud pros in securing data on AWS',
 'How to set up a G

In [9]:
def detect_language(text):
    lang, confidence = langid.classify(text)
    return lang

dataset = dataset.map(lambda example: {'Language': detect_language(example['title'] + ' ' + example['text'])})

count_english = len(dataset.filter(lambda example: example['Language'] == 'en'))
count_non_english = len(dataset.filter(lambda example: example['Language'] != 'en'))
print(f"Số lượng dòng tiếng Anh: {count_english}")
print(f"Số lượng dòng ngôn ngữ khác: {count_non_english}")
dataset = dataset.filter(lambda example: example['Language'] == 'en')

  0%|          | 0/708241 [00:00<?, ?ex/s]

  0%|          | 0/709 [00:00<?, ?ba/s]

  0%|          | 0/709 [00:00<?, ?ba/s]

Số lượng dòng tiếng Anh: 1
Số lượng dòng ngôn ngữ khác: 1


  0%|          | 0/709 [00:00<?, ?ba/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url', 'Language'],
        num_rows: 701772
    })
})

In [13]:
train_data = dataset['train']
df = pd.DataFrame(train_data)
df = df[['title', 'text']]
df.head()

,title,text
0,Daughter Duo is Dancing in The Same Company,There's a surprising twist to Regina Willoughb...
1,New York City Ballet Announces Interim Leaders...,The New York City Ballet Board of Directors an...
2,Watch Pennsylvania Ballet & Boston Ballet Face...,The Philadelphia Eagles and the New England Pa...
3,dance shoes,Looking for your next audition shoe? Shot at a...
4,Rebecca Krohn on Her Retirement from New York ...,New York City Ballet principal dancer Rebecca ...


In [22]:
df.to_csv('/kaggle/working/cc_news.csv', index=False)

# Sau khi xử lý 

In [49]:
file_path = '/kaggle/input/cc-news/cc_news.csv'
df = pd.read_csv(file_path)

In [50]:
df.head()

,title,text
0,Daughter Duo is Dancing in The Same Company,There's a surprising twist to Regina Willoughb...
1,New York City Ballet Announces Interim Leaders...,The New York City Ballet Board of Directors an...
2,Watch Pennsylvania Ballet & Boston Ballet Face...,The Philadelphia Eagles and the New England Pa...
3,dance shoes,Looking for your next audition shoe? Shot at a...
4,Rebecca Krohn on Her Retirement from New York ...,New York City Ballet principal dancer Rebecca ...


In [52]:
df.shape

(701772, 2)

In [53]:
df['text']

0         There's a surprising twist to Regina Willoughb...
1         The New York City Ballet Board of Directors an...
2         The Philadelphia Eagles and the New England Pa...
3         Looking for your next audition shoe? Shot at a...
4         New York City Ballet principal dancer Rebecca ...
                                ...                        
701767    Police are appealing for information that coul...
701768    Police are appealing for information following...
701769    Boyer + Brandy + a jar of peanut butter... Wha...
701770    No doubts that you've heard of The Golden Rule...
701771    What? How is that even possible? Blame the Gol...
Name: text, Length: 701772, dtype: object

In [55]:
total_rows_before = len(df)

# Loại bỏ hàng có giá trị null trong bất kỳ cột nào
df = df.dropna()

# Số liệu sau khi loại bỏ giá trị null
total_rows_after_null_removal = len(df)
rows_removed_null = total_rows_before - total_rows_after_null_removal

# Loại bỏ các hàng trùng lặp
df = df.drop_duplicates()

# Số liệu sau khi loại bỏ hàng trùng lặp
total_rows_after_duplicate_removal = len(df)
rows_removed_duplicates = total_rows_after_null_removal - total_rows_after_duplicate_removal

# In số liệu
print(f"Số dòng trước khi loại bỏ: {total_rows_before}")
print(f"Số dòng bị loại bỏ do giá trị null: {rows_removed_null}")
print(f"Số dòng sau khi loại bỏ giá trị null: {total_rows_after_null_removal}")
print(f"Số dòng bị loại bỏ do trùng lặp: {rows_removed_duplicates}")
print(f"Số dòng sau khi loại bỏ trùng lặp: {total_rows_after_duplicate_removal}")

Số dòng trước khi loại bỏ: 701772
Số dòng bị loại bỏ do giá trị null: 324
Số dòng sau khi loại bỏ giá trị null: 701448
Số dòng bị loại bỏ do trùng lặp: 92929
Số dòng sau khi loại bỏ trùng lặp: 608519


In [71]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Loại bỏ dấu câu
    text = re.sub(r'[^\w\s]', '', text)

    # Chuyển đổi văn bản thành chữ thường
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Loại bỏ stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Ghép lại thành câu mới
    processed_text = ' '.join(tokens)
    return processed_text

tqdm.pandas()

# Tiền xử lý toàn bộ dataset
tqdm.pandas(desc="Processing titles")
df['title'] = df['title'].progress_apply(preprocess_text)
tqdm.pandas(desc="Processing texts")
df['text'] = df['text'].progress_apply(preprocess_text)

# In ra một số ví dụ sau khi tiền xử lý
print(df['title'].head())
print(df['text'].head())

Processing texts: 100%|██████████| 608519/608519 [2:31:10<00:00, 67.09it/s]   


0                            daughter duo danc compani
1    new york citi ballet announc interim leadershi...
2    watch pennsylvania ballet boston ballet face s...
3                                            danc shoe
4             rebecca krohn retir new york citi ballet
Name: title, dtype: object
0    there surpris twist regina willoughbi last sea...
1    new york citi ballet board director announc sa...
2    philadelphia eagl new england patriot arent te...
3    look next audit shoe shot collabor broadway da...
4    new york citi ballet princip dancer rebecca kr...
Name: text, dtype: object


In [73]:
# Tạo một set chứa tất cả các từ từ cột 'title' và 'text'
all_words_set = set()

# Hàm để thực hiện tokenization, cập nhật vocab, và in thời gian
def tokenize_and_print(sentence):
    global all_words_set
    try:
        tokens = word_tokenize(sentence)
        all_words_set.update(tokens)
        return len(tokens)
    except:
        print("Skipped sentence:")
        return 0

# Bắt đầu đo thời gian
start_time = time.time()

# Xử lý cột 'title' với tqdm
tqdm.pandas(desc="Processing titles")
df['title_word_count'] = df['title'].progress_apply(tokenize_and_print)

# Xử lý cột 'text' với tqdm
tqdm.pandas(desc="Processing texts")
df['text_word_count'] = df['text'].progress_apply(tokenize_and_print)

# Kết thúc đo thời gian
end_time = time.time()

# Tính số từ trung bình
avg_title_word_count = df['title_word_count'].mean()
avg_text_word_count = df['text_word_count'].mean()

print("Số từ trung bình trong mỗi title:", avg_title_word_count)
print("Số từ trung bình trong mỗi text:", avg_text_word_count)

# Tính số lượng từ trong kho từ vựng
vocab_size = len(all_words_set)
print(f"Tổng số từ trong kho vocab: {vocab_size}")

Processing texts: 100%|██████████| 608519/608519 [25:41<00:00, 394.76it/s] 

Số từ trung bình trong mỗi title: 7.4116535391664025
Số từ trung bình trong mỗi text: 235.21719781962437
Tổng số từ trong kho vocab: 1384361


In [ ]:
df.to_csv('/kaggle/working/cc_news.csv', index=False)

# **Split data to train**

In [10]:
file_path = '/kaggle/input/cnn-news1/cc_news (2).csv'
df = pd.read_csv(file_path)
df.head()

,title,text,title_word_count,text_word_count
0,daughter duo danc compani,there surpris twist regina willoughbi last sea...,4,84
1,new york citi ballet announc interim leadershi...,new york citi ballet board director announc sa...,8,118
2,watch pennsylvania ballet boston ballet face s...,philadelphia eagl new england patriot arent te...,8,93
3,danc shoe,look next audit shoe shot collabor broadway da...,2,45
4,rebecca krohn retir new york citi ballet,new york citi ballet princip dancer rebecca kr...,7,442


In [11]:
df.shape

(608519, 4)

In [13]:
# Lấy 50% dữ liệu từ df
df_sample = df.sample(frac=0.33, random_state=42)

# Chia thành tập train (75%), test (20%) và validation (5%)
train_df, temp_df = train_test_split(df_sample, test_size=0.25, random_state=42)
test_df, validation_df = train_test_split(temp_df, test_size=0.2, random_state=42)

# Kiểm tra kích thước của từng tập
print("Kích thước tập train:", len(train_df))
print("Kích thước tập test:", len(test_df))
print("Kích thước tập validation:", len(validation_df))

Kích thước tập train: 150608
Kích thước tập test: 40162
Kích thước tập validation: 10041


In [14]:
test_df.head()

,title,text,title_word_count,text_word_count
156096,marka launch new retail concept,statement dubai bours marka said anoth 10 outl...,5,18
105368,lebron miss 2nd exhibit game sprain ankl,cleveland lebron jame sit friday exhibit game ...,7,68
278273,research find link brain structur hallucin pro...,new research publish schizophrenia research co...,9,158
549965,punish unlik judg call convict rapist good man...,salt lake citi ap complaint keep pour utah jud...,9,313
191545,soul calibur 6 releas date charact roster gues...,bandai namco step back onto stage histori here...,13,572


In [ ]:
# Hàm để tạo và cấu hình mô hình GPT-2
def create_gpt2_model():
    # Cấu hình mô hình GPT-2
    config = GPT2Config.from_pretrained("gpt2")
    
    # Tạo mô hình GPT-2 với 345M parameters
    model = GPT2LMHeadModel(config)
    
    return model

# Hàm để chuyển đổi dữ liệu thành định dạng text-to-text
def create_text_to_text_data(dataframe):
    text_to_text_data = [f"{text} TL;DR: {title} <eos>" for text, title in zip(dataframe['text'], dataframe['title'])]
    return text_to_text_data

# Class Dataset để chuẩn bị dữ liệu cho fine-tuning
class TextToTextDataset(Dataset):
    def __init__(self, text_to_text_data, tokenizer):
        self.text_to_text_data = text_to_text_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.text_to_text_data)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.text_to_text_data[idx], truncation=True, padding=True, return_tensors="pt")
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": encoding["input_ids"].flatten(),  # Label là input (auto-regressive)
        }

# Hàm để fine-tune mô hình GPT-2
def fine_tune_gpt2(model, train_dataset, val_dataset, epochs=3, learning_rate=2e-05):
    # Di chuyển mô hình lên GPU nếu có
    model.to("cuda")

    # Tạo DataLoader cho tập huấn luyện và tập validation
    train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=custom_collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=custom_collate_fn)

    # Loss function và optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    
    total_steps = len(train_dataloader) * epochs
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)

    # Fine-tuning
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        model.train()
        for batch_idx, batch in enumerate(train_dataloader):
            input_ids = batch["input_ids"].to("cuda")
            attention_mask = batch["attention_mask"].to("cuda")
            labels = batch["labels"].to("cuda")

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx}/{len(train_dataloader)}, Loss: {loss.item()}")

        # Đánh giá mô hình trên tập validation
        model.eval()
        total_loss = 0.0
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_dataloader):
                input_ids = batch["input_ids"].to("cuda")
                attention_mask = batch["attention_mask"].to("cuda")
                labels = batch["labels"].to("cuda")

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                total_loss += outputs.loss.item()

                if batch_idx % 10 == 0:
                    print(f"Validation Batch {batch_idx}/{len(val_dataloader)}, Loss: {outputs.loss.item()}")

        avg_loss = total_loss / len(val_dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_loss}")
    return model

# Hàm collate_fn tùy chỉnh để xử lý độ dài không đồng đều của các mẫu trong batch
def custom_collate_fn(batch):
    # Sắp xếp batch theo độ dài giảm dần
    sorted_batch = sorted(batch, key=lambda x: x['input_ids'].shape[0], reverse=True)

    # Lấy kích thước của tensor lớn nhất
    max_len = sorted_batch[0]['input_ids'].shape[0]

    # Padding tất cả các tensor để chúng có kích thước giống nhau
    padded_batch = {
        'input_ids': torch.stack([F.pad(x['input_ids'], (0, max_len - x['input_ids'].shape[0])) for x in sorted_batch]),
        'attention_mask': torch.stack([F.pad(x['attention_mask'], (0, max_len - x['attention_mask'].shape[0])) for x in sorted_batch]),
        'labels': torch.stack([F.pad(x['labels'], (0, max_len - x['labels'].shape[0])) for x in sorted_batch])
    }

    return padded_batch

# Tạo mô hình GPT-2
gpt2_model = create_gpt2_model()

# Tạo dữ liệu
train_text_to_text_data = create_text_to_text_data(train_df)
val_text_to_text_data = create_text_to_text_data(validation_df)
test_text_to_text_data = create_text_to_text_data(test_df)

# Tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tạo Dataset
train_dataset = TextToTextDataset(train_text_to_text_data, tokenizer)
test_dataset = TextToTextDataset(test_text_to_text_data, tokenizer)
val_dataset = TextToTextDataset(val_text_to_text_data, tokenizer)

print("Training set size:", len(train_dataset))
print("Testing set size:", len(test_dataset))
print("Validation set size:", len(val_dataset))

# Fine-tune mô hình
fine_tuned_model = fine_tune_gpt2(gpt2_model, train_dataset, val_dataset)

In [ ]:
model_save_path = "/kaggle/working/fine_tuned_gpt2_model"
tokenizer.save_vocabulary("/kaggle/working")
fine_tuned_model.save_pretrained(model_save_path)

In [15]:
vocab_path = os.path.join("/kaggle/input/vocab-token-news", "vocab.json")
merges_path = os.path.join("/kaggle/input/vocab-token-news", "merges.txt")
#vocab_path = os.path.join("/kaggle/working/", "vocab.json")
#merges_path = os.path.join("/kaggle/working/", "merges.txt")
loaded_tokenizer = GPT2TokenizerFast(vocab_file=vocab_path, merges_file=merges_path)
loaded_tokenizer.add_special_tokens({'pad_token': loaded_tokenizer.eos_token})
# Tạo mô hình từ thư mục đã lưu mô hình
model_save_path = "/kaggle/input/fine-tuned-gpt2-model-news"
#model_save_path = "/kaggle/working/fine_tuned_gpt2_model"
loaded_model = GPT2LMHeadModel.from_pretrained(model_save_path)

In [52]:
def generate_title(model, text, tokenizer, top_k=250, top_p=0.6):
    # Tokenize and convert to tensor
    input_text = f"{text} TL;DR:"
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True)["input_ids"]

    # Move tensor to the same device as the model
    input_ids = input_ids.to(model.device)
    max_length = len(tokenizer.tokenize(input_text)) + 15

    # Generate title
    output = model.generate(
        input_ids,
        pad_token_id=tokenizer.eos_token_id,
        max_length = max_length,
        top_k=top_k,
        top_p=top_p,
        temperature=0.7,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        attention_mask=input_ids.ne(tokenizer.pad_token_id).long(),
    )

    # Move the result back to CPU
    output = output.to("cpu")

    # Decode and extract the text after "TL;DR:"
    generated_title = tokenizer.decode(output[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    # Tìm vị trí của token <eos> trong chuỗi sinh ra
    eos_index = generated_title.find("<eos>")
    
    # Nếu token <eos> được tìm thấy, cắt chuỗi tại vị trí đó
    if eos_index != -1:
        generated_title = generated_title[:eos_index].strip()

    title_start_index = generated_title.find("TL;DR:") + len("TL;DR:")
    extracted_title = generated_title[title_start_index:].strip()

    return extracted_title

In [53]:
def selection_module(text, candidate_titles):
    # Component 1: Exact Match
    for title in candidate_titles:
        if title in text:
            return title
    # Component 2: Noun Phrase Overlap
    noun_phrase_scores = {}
    for title in candidate_titles:
        noun_phrase_overlap = calculate_noun_phrase_overlap(title, text)
        noun_phrase_scores[title] = noun_phrase_overlap

    max_noun_phrase_title = max(noun_phrase_scores, key=noun_phrase_scores.get)
    if noun_phrase_scores[max_noun_phrase_title] >= 2:
        return max_noun_phrase_title
    # Component 3: Semantic Similarity
    semantic_scores = {}
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can choose a different pre-trained model
    text_embedding = model.encode(text)

    for title in candidate_titles:
        title_embedding = model.encode(title)
        semantic_similarity = cosine_similarity([title_embedding], [text_embedding])[0][0]
        semantic_scores[title] = semantic_similarity

    selected_title = max(semantic_scores, key=semantic_scores.get)
    return selected_title
def calculate_noun_phrase_overlap(title, text):
    # Tokenize and tag parts of speech
    title_tags = pos_tag(word_tokenize(title))
    text_tags = pos_tag(word_tokenize(text))

    # Extract Noun Phrases
    title_noun_phrases = [word for word, pos in title_tags if pos.startswith('NN')]
    text_noun_phrases = [word for word, pos in text_tags if pos.startswith('NN')]

    # Calculate overlap
    overlap_count = len(set(title_noun_phrases) & set(text_noun_phrases))
    return overlap_count

In [54]:
test_df.iloc[2]['text']

'new research publish schizophrenia research conduct univers liverpool link brain structur individu likelihood experienc hallucin music aptitud previou research show musician increas white matter integr specif part brain call corpu callosum thick band nerv fiber connect left right halv brain enabl commun hemispher psychot individu auditori verbal hallucin integr corpu callosum found reduc research univers psycholog scienc depart identifi 38 healthi individu age 18 63 test propens hallucin music aptitud measur detail brain structur use mri scanner research observ particip higher music aptitud show lower hallucin prone importantli research reveal music aptitud posit associ corpu callosum integr wherea hallucin prone associ lower integr fibr connect two hemispher brain statist analysi indic relationship hallucin prone music aptitud mediat microstructur corpu callosum studi research ami spray said result could import clinic implic music aptitud increas white matter integr corpu callosum mu

In [55]:
text = test_df.iloc[2]['text']
title = test_df.iloc[2]['title']
candidate_titles = [generate_title(loaded_model, text, loaded_tokenizer) for _ in range(10)]
generation_title = random.choice(candidate_titles)
print("Text: %s" %(text))
print("Actual Titile: %s" %(title))
print("Generation Titile: %s" %(generation_title))
for candidate_title in candidate_titles:
    print("Title: %s" %(candidate_title))
selected_title = selection_module(text, candidate_titles)
print("Selection Titile: %s" %(selected_title))

Text: new research publish schizophrenia research conduct univers liverpool link brain structur individu likelihood experienc hallucin music aptitud previou research show musician increas white matter integr specif part brain call corpu callosum thick band nerv fiber connect left right halv brain enabl commun hemispher psychot individu auditori verbal hallucin integr corpu callosum found reduc research univers psycholog scienc depart identifi 38 healthi individu age 18 63 test propens hallucin music aptitud measur detail brain structur use mri scanner research observ particip higher music aptitud show lower hallucin prone importantli research reveal music aptitud posit associ corpu callosum integr wherea hallucin prone associ lower integr fibr connect two hemispher brain statist analysi indic relationship hallucin prone music aptitud mediat microstructur corpu callosum studi research ami spray said result could import clinic implic music aptitud increas white matter integr corpu callos

In [57]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU()

score = bleu_scorer.sentence_score(
    hypothesis=generation_title,
    references=[title],
)
print("BLEU Score - Generation Title:", score.score/100)

score = bleu_scorer.sentence_score(
    hypothesis=selected_title,
    references=[title],
)
print("BLEU Score - Selection Title:", score.score/100)

BLEU Score - Generation Title: 0.0
BLEU Score - Selection Title: 0.05442414219118318
